In [1]:
using BenchmarkTools
using Combinatorics

include("../phase1/node.jl")
include("../phase1/edges.jl")
include("../phase1/graph.jl")
include("../phase1/read_stsp.jl")
include("../phase2/comp_connexes.jl")
include("../phase1/main.jl")
include("../phase2/queue.jl")
include("../phase2/heuristics.jl")
include("../phase2/Kruskal.jl")
include("../phase2/prims_algorithm.jl")

prims_algorithm

In [2]:
tree_root = TreeNode("1", 0)
child1 = TreeNode("2", 0)
child2 = TreeNode("3", 0)
child3 = TreeNode("4", 0)
child4 = TreeNode("5",0)
child5 = TreeNode("6",0)
child6 = TreeNode("7",0)

tree = Tree("test tree", Vector{TreeNode{Int}}())

first_row = [ child1, child2, child3]
second_row = [child4, child6]
add_node!(tree, tree_root)
for child in first_row
    add_node!(tree, child, parent = tree_root)
end
for child in second_row
    add_node!(tree, child, parent = child3)
end
add_node!(tree, child5, parent = child4)
show(tree)

Node 1 has  rank 0 and 3 children.
It has no parent.
Its children are: 
     2
     3
     4
Node 2 has  rank 0 and 0 children.
Its parent is 1
It has no children.
Node 3 has  rank 0 and 0 children.
Its parent is 1
It has no children.
Node 4 has  rank 0 and 2 children.
Its parent is 1
Its children are: 
     5
     7
Node 5 has  rank 0 and 1 children.
Its parent is 4
Its children are: 
     6
Node 7 has  rank 0 and 0 children.
Its parent is 4
It has no children.
Node 6 has  rank 0 and 0 children.
Its parent is 5
It has no children.


In [3]:
n1= TreeNode("1", 0)
n2 = TreeNode("2", 0)
n3 = TreeNode("3", 0)
n4= TreeNode("4", 0)
t1 = Tree("test tree" , Vector{TreeNode{Int}}())
t1 = add_node!(t1, n1)
t1 = add_node!(t1, n2, parent = n1)
t1 = add_node!(t1, n3)
t1 = add_node!(t1, n4, parent = n3)
rank_union!(t1, n1, n3)


TreeNode{Int64}("1", 0, nothing, [2, 3], 1, 1)

In [4]:
show(t1)

Node 1 has  rank 1 and 2 children.
It has no parent.
Its children are: 
     2
     3
Node 2 has  rank 0 and 0 children.
Its parent is 1
It has no children.
Node 3 has  rank 0 and 1 children.
Its parent is 1
Its children are: 
     4
Node 4 has  rank 0 and 0 children.
Its parent is 3
It has no children.


In [5]:
"""crée une liste d'adjacence a partir d'un graphe"""
function adjacency_dict( graph::Graph) 
    #Dictionaire de adjacence
    adj_dict = Dict()
    #Dictionaire de correspondance entre les noeuds et les items de la file de priorite
    correspondance_dict = Dict()
    #Chaque noeud est une clef du dictionaire de adjacence
    for (i, node) in enumerate(nodes(graph))
        adj_dict[i] = Dict()
        correspondance_dict[name(node)] = i
    end
    #Ajoute les voisins de chaque noeud dans le dictionaire de adjacence
    for edge in edges(graph)
        node1, node2 = nodes(edge)
        idx1 = correspondance_dict[name(node1)]
        idx2 = correspondance_dict[name(node2)]
        adj_dict[idx1][idx2] = weight(edge)
        adj_dict[idx2][idx1] = weight(edge)
    end
    return adj_dict
end

"""Crée une file de priorite des arbres à partir d'un graphe. Le noeud de depart a une priorite de 0 et les autres ont une priorite de Inf"""
function prims_priority_queue(graph::Graph{Y,T}, start_node_name::String) where {Y,T}
    priority_queue = PriorityQueue{PriorityItem{TreeNode{Y}}}()
    tree = Tree(name(graph), Vector{TreeNode{Y}}())
    for node in nodes(graph)
        if name(node) == start_node_name
            blank_node = TreeNode(name(node), data(node))
            priority_item = PriorityItem( 0 , blank_node)
        else
            blank_node = TreeNode(name(node), data(node))
            priority_item = PriorityItem( Inf, blank_node)
        end
        add_node!(tree, blank_node)
        push!(priority_queue, priority_item)
    end
    return priority_queue, tree
end

"""Implementation de l'algorithme de Prim"""
function prims_algorithm(graph::Graph{Y,T}; start_node_name::Any = nothing) where {Y,T}
    #initialisation
    if isnothing(start_node_name)
        start_node_name = name(nodes(graph)[1])
    end
    #initialisation de la file de priorite et du dictionaire d'adjacence
    priority_queue, tree = prims_priority_queue(graph, start_node_name)
    adjacency_list = adjacency_dict(graph)
    #sauvegarde de la racine
    root = poplast!(priority_queue)
    priority_node = root
    #boucle principale
    while !is_empty(priority_queue)
        for  (neighbor_idx, edge_weight) in adjacency_list[index(data(priority_node))]
            #Met a jour la priorite du voisin si elle est dans le priority_que et elle est plus petite que la priorite actuelle
            for item in priority_queue.items
                #println("nodes(tree)[neighbor_idx] = ", nodes(tree)[neighbor_idx])
                if index(data(item)) == neighbor_idx
                    if  edge_weight < priority(item)
                        change_parent!(tree, nodes(tree)[neighbor_idx], data(priority_node))
                        change_rank!( nodes(tree)[neighbor_idx], edge_weight)
                        priority!(item, edge_weight)
                    end
                break
                end
            end
        end
        #prend le noeud avec la plus petite priorite
        priority_node = poplast!(priority_queue)         
    end
    return tree, data(root)
end

prims_algorithm

In [6]:
graphe_test = Graph("Test",Node{Vector{Float64}}[],Edge{Int,Vector{Float64}}[])

#Nodes 
nodea = Node("a",[0.])
nodeb = Node("b",[0.])
nodec = Node("c",[0.])
noded = Node("d",[0.])
nodee = Node("e",[0.])
nodef = Node("f",[0.])
nodeg = Node("g",[0.])
nodeh = Node("h",[0.])
nodei = Node("i",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee, nodef, 
                nodeg, nodeh, nodei]

#Edges
edge1 = Edge(nodea,nodeb, 4)
edge2 = Edge(nodea,nodeh, 8)
edge3 = Edge(nodeb, nodeh, 11)
edge4 = Edge(nodeb, nodec, 8)
edge5 = Edge(nodeh, nodei, 7)
edge6 = Edge(nodeh, nodeg, 1)
edge7 = Edge(nodeg, nodei, 6)
edge8 = Edge(nodeg, nodef, 2)
edge9 = Edge(nodec, nodef, 4)
edge10 = Edge(nodei, nodec, 2)
edge11 = Edge(nodef, nodee, 10)
edge12 = Edge(nodee, noded, 9)
edge13 = Edge(nodec,noded, 7)
edge14 = Edge(noded, nodef, 14)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10, edge11,
                edge12, edge13, edge14]

##Construction du graphe
for i in node_list
    add_node!(graphe_test,i)
end
for i in edge_list
    add_edge!(graphe_test, i)
end


In [7]:
priority_queue, tree = prims_priority_queue(graphe_test, "a")

#@benchmark p_tree, root = prims_algorithm(graphe_test, start_node_name = "a")
kruskal_tree = kruskal(graphe_test)

UndefVarError: UndefVarError: `treenodes_list` not defined

In [8]:
#@benchmark kruskal(graphe_test)

In [9]:
@time p_tree1, root1 = prims_algorithm(graphe_test, start_node_name = "a")

  0.175259 seconds (365.94 k allocations: 24.580 MiB, 3.96% gc time, 99.33% compilation time)


(Tree{Vector{Float64}}("Test", TreeNode{Vector{Float64}}[TreeNode{Vector{Float64}}("a", [0.0], nothing, [2], 0, 1), TreeNode{Vector{Float64}}("b", [0.0], 1, [3], 4, 2), TreeNode{Vector{Float64}}("c", [0.0], 2, [4, 6, 9], 8, 3), TreeNode{Vector{Float64}}("d", [0.0], 3, [5], 7, 4), TreeNode{Vector{Float64}}("e", [0.0], 4, Int64[], 9, 5), TreeNode{Vector{Float64}}("f", [0.0], 3, [7], 4, 6), TreeNode{Vector{Float64}}("g", [0.0], 6, [8], 2, 7), TreeNode{Vector{Float64}}("h", [0.0], 7, Int64[], 1, 8), TreeNode{Vector{Float64}}("i", [0.0], 3, Int64[], 2, 9)]), TreeNode{Vector{Float64}}("a", [0.0], nothing, [2], 0, 1))

In [10]:
swiss_42_graph, swiss_42_nodes = graph_from_tsp("../../instances/stsp/swiss42.tsp","graphe1")
println("running Prim's algorithm on swiss42")
@time swiss_42_tree, root2 = prims_algorithm(swiss_42_graph)
swiss_42_graph = tree_to_graph(swiss_42_tree, root2)
sum_of_weights(swiss_42_graph)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
running Prim's algorithm on swiss42
  0.036309 seconds (60.90 k allocations: 3.488 MiB, 95.41% compilation time)


1079.0

In [11]:
graphe1, graphe1_nodes = graph_from_tsp("../../instances/stsp/bays29.tsp","graphe1")
println("running Prim's algorithm")
@time bays_29_tree, root3 = prims_algorithm(graphe1)
bays_29_graph = tree_to_graph(bays_29_tree, root3)

sum_of_weights(bays_29_graph)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
running Prim's algorithm
  0.000576 seconds (6.96 k allocations: 253.953 KiB)


1557.0

## Algorithm de Held et Karp 

In [12]:
# nodea = Node("a",[0.])
# nodeb = Node("b",[0.])
# nodec = Node("c",[0.])
# noded = Node("d",[0.])
# nodee = Node("e",[0.])
# node_list = [nodea,nodeb,nodec, noded,nodee]
# #Edges are fully connected
# edge1 = Edge(nodea,nodeb, 4.)
# edge2 = Edge(nodea,nodec, 8.)
# edge3 = Edge(nodea, noded, 11.)
# edge4 = Edge(nodea, nodee, 8.)
# edge5 = Edge(nodeb, nodec, 7.)
# edge6 = Edge(nodeb, noded, 1.)
# edge7 = Edge(nodeb, nodee, 6.)
# edge8 = Edge(nodec, noded, 2.)
# edge9 = Edge(nodec, nodee, 4.)
# edge10 = Edge(noded, nodee, 7.)
# edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
#                 edge7, edge8, edge9, edge10]

# tsp_test = Graph("Test",node_list,edge_list)
# tsp_test

nodea = Node("a",[0.])
nodeb = Node("b",[0.])
nodec = Node("c",[0.])
noded = Node("d",[0.])
nodee = Node("e",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee]
#Edges are fully connected
edge1 = Edge(nodea,nodeb, 3.)
edge2 = Edge(nodea,nodec, 2.)
edge3 = Edge(nodea, noded, 11.)
edge4 = Edge(nodea, nodee, 9.)
edge5 = Edge(nodeb, nodec, 7.)
edge6 = Edge(nodeb, noded, 12.)
edge7 = Edge(nodeb, nodee, 6.)
edge8 = Edge(nodec, noded, 6.)
edge9 = Edge(nodec, nodee, 4.)
edge10 = Edge(noded, nodee, 7.)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10]

tsp_test = Graph("Test",node_list,edge_list)
tsp_test



Graph{Vector{Float64}, Float64}("Test", Node{Vector{Float64}}[Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("c", [0.0]), Node{Vector{Float64}}("d", [0.0]), Node{Vector{Float64}}("e", [0.0])], Edge{Float64, Vector{Float64}}[Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), 3.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("c", [0.0]), 2.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("d", [0.0]), 11.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("e", [0.0]), 9.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("c", [0.0]), 7.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("d", [0.0]), 12.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}

In [13]:
function h_k_exact_algorithm(g::Graph ; start_node_name::Any = nothing)
    #if no start node name, first node is start node
    if isnothing(start_node_name)
        start_node_name = name(nodes(g)[1])
    end
    #move start node to first position of graph
    for (i, node) in enumerate(nodes(g))
        if name(node) == start_node_name
            g.nodes[1], g.nodes[i] = g.nodes[i], g.nodes[1]
            break
        end
    end
    #create adjacency list
    adjacency_list = adjacency_dict(g)
    distance_dict = Dict()
    for k in 2:length(nodes(g))
        distance_dict[(Set([k]), k)] = Dict("c" =>adjacency_list[1][k], "p"=> [1, k])
    end
    for s in 2:length(nodes(g))-1
        for subset in combinations(2:length(nodes(g)), s)
            for k in subset
                min_dist = Inf
                for m in subset 
                    if m !=k
                        predecessor = distance_dict[(setdiff(Set(subset), Set([k])), m)]
                        #println("predecessor", predecessor)
                        dist = predecessor["c"] + adjacency_list[m][k]
                        if dist < min_dist
                            min_dist = dist
                            min_m = m
                            new_list = copy(predecessor["p"])
                            append!(new_list, k)
                            distance_dict[(Set(subset), k)] = Dict("c" => min_dist, "p" => new_list)
                        end
                    end
                end
            end
        end
    end

    #returns the minimum path from the start node that goes to all of the other paths
    min = Inf
    final_path = []
    for k in 2:length(nodes(g))
        distance = distance_dict[(Set(2:length(nodes(g))), k)]["c"] + adjacency_list[k][1]
        println("distance = ", distance)
        if distance < min
            println("distance = ", distance)
            min = distance
            final_path = copy(distance_dict[(Set(2:length(nodes(g))), k)]["p"])
            append!(final_path, 1)
        end
    end
    return min, final_path
end


h_k_algorithm(tsp_test)


UndefVarError: UndefVarError: `h_k_algorithm` not defined

In [14]:


function get_shortest_edges(graph::Graph, departure_node::Node)
    shortest_edge_vec = Vector{Edge}(undef, 2)
    shortest_dist_vec = Vector{Float64}([Inf, Inf])
    for  edge in edges(graph)
        node1, node2 = nodes(edge)
        if name(node1) == name(departure_node) || name(node2) == name(departure_node)
            #copies over the previous best to second best
            if weight(edge) < shortest_dist_vec[1]
                if shortest_dist_vec[1]< shortest_dist_vec[2]
                    shortest_dist_vec[2] = shortest_dist_vec[1]
                    shortest_edge_vec[2] = shortest_edge_vec[1]
                end
                shortest_dist_vec[1] = weight(edge)
                shortest_edge_vec[1] = edge
            elseif weight(edge) < shortest_dist_vec[2]
                shortest_dist_vec[2] = weight(edge)
                shortest_edge_vec[2] = edge
            end
        end
    end
    return  shortest_edge_vec[1], shortest_edge_vec[2]
end



function find_one_tree(graph::Graph, departure_node::Node, adjacency_list::Dict)
    #Copies graph to feed into prim's algorithm
    start_graph = deepcopy(graph)
    #removes the departure node from the graph and saves its index
    idx = remove_node!(start_graph, departure_node)
    remove_edges!(start_graph, departure_node)
    #uses prims algorithm to find the one_tree
    one_tree, root = prims_algorithm(start_graph, start_node_name = name(departure_node))
    #converts the prims algorithm to a graph
    one_tree_graph = tree_to_graph(one_tree, root)
    #finds the second shortest edge from the departure node
    shortest_edge_1, shortest_edge_2   = get_shortest_edges(graph, departure_node)
    add_node!(one_tree_graph, departure_node, idx)
    add_edge!(one_tree_graph,shortest_edge_1)
    add_edge!(one_tree_graph,shortest_edge_2)
    one_tree_distance = sum_of_weights(one_tree_graph)
    return one_tree_distance, one_tree_graph
end

#Returns the degrees of nodes in  a graph
function degree(graph::Graph)
    adjacency_list = adjacency_dict(graph)
    degrees = []
    for i in 1:length(nodes(graph))
        push!(degrees, length(keys(adjacency_list[i])))
    end
    return degrees
end

function update_edge_weights!(graph::Graph, pis::Vector{Float64})
    correspondance_dict = Dict()
    for (i, node) in enumerate(nodes(graph))
        correspondance_dict[name(node)] = i
    end
    for edge in edges(graph)
        node1, node2 = nodes(edge)
        node1_idx = correspondance_dict[name(node1)]
        node2_idx = correspondance_dict[name(node2)]
        new_weight = weight(edge) + pis[node1_idx] + pis[node2_idx]
        set_weight!(edge, new_weight)
    end
end

"""subgradient heuristic for calculating a minimal tour"""
function lkh_subgradient(start_graph::Graph;  departure_node::Union{Node, Nothing} = nothing, t_k_method::String = "1/k")
    
    no_nodes = length(nodes(start_graph))
    if isnothing(departure_node)
        departure_node = nodes(start_graph)[1]
    end
    graph = deepcopy(start_graph)
    k = 0
    w = -Inf
    pis = zeros(length(nodes(graph)))
    adjacency_list = adjacency_dict(graph)

    while true && k < 10000000
        iter_time = time()
        total_distance, one_tree = find_one_tree(graph, departure_node, adjacency_list)
        weights_k = total_distance - 2 * sum(pis)
        w = max(w, weights_k)
        v_k = degree(one_tree) .- 2
        #Calculates the l1 norm of v_k
        
        if v_k == zeros(length(nodes(graph)))
            return total_distance, one_tree
        end
        if t_k_method == "1/k"
            t_k = 1/(k+1)
        elseif t_k_method == "blah"
            t_k = 1
        else
            start_weight = sum_of_weights(start_graph)
            t_k = start_weight/(100*no_nodes + k)
        end
        pis = pis + t_k * v_k
        k = k + 1
        update_edge_weights!(graph, pis)
        v_k_norm  = sum(broadcast(abs, v_k))
        println("k = ", k, " time = ", time() - iter_time, " VK_norm ", v_k_norm )
    end
    return Inf, nothing
end

lkh_subgradient

In [15]:
function get_leaves(graph::Graph)
    leaves = []
    adj_dict = adjacency_dict(graph)
    for node in keys(adj_dict)
        if length(keys(adj_dict[node])) == 1
            push!(leaves, node)
        end
    end
    return leaves
end


get_leaves (generic function with 1 method)

In [16]:
p_tree1, root1 = prims_algorithm(tsp_test, start_node_name = "a")
show(p_tree1)
p_graph = tree_to_graph( p_tree1, root1)
get_leaves(p_graph)

Node a has  rank 0 and 2 children.
It has no parent.
Its children are: 
     b
     c
Node b has  rank 3.0 and 0 children.
Its parent is a
It has no children.
Node c has  rank 2.0 and 2 children.
Its parent is a
Its children are: 
     e
     d
Node d has  rank 6.0 and 0 children.
Its parent is c
It has no children.
Node e has  rank 4.0 and 0 children.
Its parent is c
It has no children.


3-element Vector{Any}:
 5
 4
 2

In [17]:
@time lkh_subgradient(tsp_test, departure_node = nodec, t_k_method = "1/k")

k = 1 time = 0.04905986785888672 VK_norm 2
  0.175443 seconds (533.87 k allocations: 35.970 MiB, 3.26% gc time, 98.43% compilation time)


(24.0, Graph{Vector{Float64}, Float64}("Test", Node{Vector{Float64}}[Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("c", [0.0]), Node{Vector{Float64}}("d", [0.0]), Node{Vector{Float64}}("e", [0.0])], Edge{Float64, Vector{Float64}}[Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), 3.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("e", [0.0]), Node{Vector{Float64}}("d", [0.0]), 7.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("e", [0.0]), 7.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("c", [0.0]), 2.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("c", [0.0]), Node{Vector{Float64}}("d", [0.0]), 5.0)]))

In [18]:

#Fully connected graph with 7 edges
a = Node("a",[0.])
b = Node("b",[0.])
c = Node("c",[0.])
d = Node("d",[0.])
e = Node("e",[0.])
f = Node("f",[0.])
g = Node("g",[0.])
node_list = [a,b,c, d,e, f, g]
#Edges are fully connected
edge1 = Edge(a,b, 4.)
edge2 = Edge(a,c, 8.)
edge3 = Edge(a, d, 11.)
edge4 = Edge(a, e, 8.)
edge5 = Edge(a, f, 7.)
edge6 = Edge(a, g, 1.)
edge7 = Edge(b, c, 6.)
edge8 = Edge(b, d, 2.)
edge9 = Edge(b, e, 4.)
edge10 = Edge(b, f, 7.)
edge11 = Edge(b, g, 2.)
edge12 = Edge(c, d, 7.)
edge13 = Edge(c, e, 1.)
edge14 = Edge(c, f, 6.)
edge15 = Edge(c, g, 3.)
edge16 = Edge(d, e, 5.)
edge17 = Edge(d, f, 4.)
edge18 = Edge(d, g, 8.)
edge19 = Edge(e, f, 2.)
edge20 = Edge(e, g, 7.)
edge21 = Edge(f, g, 3.)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10, edge11,
                edge12, edge13, edge14, edge15, edge16,
                edge17, edge18, edge19, edge20, edge21]
#creates the graph
tsp_test2 = Graph("Test2",node_list,edge_list)

score, test2_graph = lkh_subgradient(tsp_test2, t_k_method = "1/k")
score

k = 1 time = 0.00011205673217773438 VK_norm 4
k = 2 time = 9.298324584960938e-5 VK_norm 2
k = 3 time = 7.605552673339844e-5 VK_norm 4
k = 4 time = 7.295608520507812e-5 VK_norm 4
k = 5 time = 7.510185241699219e-5 VK_norm 6
k = 6 time = 7.486343383789062e-5 VK_norm 6
k = 7 time = 6.890296936035156e-5 VK_norm 8
k = 8 time = 6.794929504394531e-5 VK_norm 8
k = 9 time = 7.200241088867188e-5 VK_norm 6
k = 10 time = 7.605552673339844e-5 VK_norm 6
k = 11 time = 6.794929504394531e-5 VK_norm 6
k = 12 time = 6.794929504394531e-5 VK_norm 4
k = 13 time = 6.794929504394531e-5 VK_norm 4
k = 14 time = 6.079673767089844e-5 VK_norm 4
k = 15 time = 6.794929504394531e-5 VK_norm 4
k = 16 time = 7.486343383789062e-5 VK_norm 4
k = 17 time = 6.198883056640625e-5 VK_norm 4
k = 18 time = 5.91278076171875e-5 VK_norm 4
k = 19 time = 5.698204040527344e-5 VK_norm 4
k = 20 time = 6.604194641113281e-5 VK_norm 6
k = 21 time = 8.511543273925781e-5 VK_norm 6
k = 22 time = 8.0108642578125e-5 VK_norm 6
k = 23 time = 7.3909

 VK_norm 8
k = 228 time = 7.295608520507812e-5 VK_norm 8
k = 229 time = 6.198883056640625e-5 VK_norm 8
k = 230 time = 5.817413330078125e-5 VK_norm 8
k = 231 time = 6.29425048828125e-5 VK_norm 8
k = 232 time = 5.602836608886719e-5 VK_norm 8
k = 233 time = 6.604194641113281e-5 VK_norm 8
k = 234 time = 6.103515625e-5 VK_norm 8
k = 235 time = 5.698204040527344e-5 VK_norm 8
k = 236 time = 6.890296936035156e-5 VK_norm 8
k = 237 time = 5.698204040527344e-5 VK_norm 8
k = 238 time = 5.602836608886719e-5 VK_norm 8
k = 239 time = 6.29425048828125e-5 VK_norm 8
k = 240 time = 6.008148193359375e-5 VK_norm 8
k = 241 time = 5.793571472167969e-5 VK_norm 8
k = 242 time = 5.793571472167969e-5 VK_norm 8
k = 243 time = 5.698204040527344e-5 VK_norm 8
k = 244 time = 6.198883056640625e-5 VK_norm 8
k = 245 time = 5.817413330078125e-5 VK_norm 8
k = 246 time = 5.602836608886719e-5 VK_norm 8
k = 247 time = 6.198883056640625e-5 VK_norm 8
k = 248 time = 5.602836608886719e-5 VK_norm 8
k = 249 time = 6.103515625e-5 V

316 time = 6.914138793945312e-5 VK_norm 8
k = 317 time = 7.915496826171875e-5 VK_norm 8
k = 318 time = 7.009506225585938e-5 VK_norm 8
k = 319 time = 7.510185241699219e-5 VK_norm 8
k = 320 time = 6.794929504394531e-5 VK_norm 8
k = 321 time = 6.890296936035156e-5 VK_norm 8
k = 322 time = 6.699562072753906e-5 VK_norm 8
k = 323 time = 5.817413330078125e-5 VK_norm 8
k = 324 time = 7.200241088867188e-5 VK_norm 8
k = 325 time = 5.602836608886719e-5 VK_norm 8
k = 326 time = 5.698204040527344e-5 VK_norm 8
k = 327 time = 5.5789947509765625e-5 VK_norm 8
k = 328 time = 5.602836608886719e-5 VK_norm 8
k = 329 time = 7.987022399902344e-5 VK_norm 8
k = 330 time = 7.104873657226562e-5 VK_norm 8
k = 331 time = 6.890296936035156e-5 VK_norm 8
k = 332 time = 7.486343383789062e-5 VK_norm 6
k = 333 time = 6.890296936035156e-5 VK_norm 6
k = 334 time = 6.890296936035156e-5 VK_norm 6
k = 335 time = 5.793571472167969e-5 VK_norm 6
k = 336 time = 5.698204040527344e-5 VK_norm 6
k = 337 time = 5.984306335449219e-5 V


k = 540 time = 7.295608520507812e-5 VK_norm 8
k = 541 time = 7.796287536621094e-5 VK_norm 8
k = 542 time = 6.818771362304688e-5 VK_norm 8
k = 543 time = 7.510185241699219e-5 VK_norm 8
k = 544 time = 5.888938903808594e-5 VK_norm 8
k = 545 time = 5.7220458984375e-5 VK_norm 8
k = 546 time = 6.29425048828125e-5 VK_norm 8
k = 547 time = 5.698204040527344e-5 VK_norm 8
k = 548 time = 5.698204040527344e-5 VK_norm 6
k = 549 time = 5.698204040527344e-5 VK_norm 6
k = 550 time = 5.602836608886719e-5 VK_norm 6
k = 551 time = 6.198883056640625e-5 VK_norm 6
k = 552 time = 5.698204040527344e-5 VK_norm 6
k = 553 time = 6.985664367675781e-5 VK_norm 6
k = 554 time = 8.392333984375e-5 VK_norm 6
k = 555 time = 6.818771362304688e-5 VK_norm 6
k = 556 time = 7.104873657226562e-5 VK_norm 6
k = 557 time = 6.699562072753906e-5 VK_norm 6
k = 558 time = 6.699562072753906e-5 VK_norm 6
k = 559 time = 7.200241088867188e-5 VK_norm 6
k = 560 time = 6.985664367675781e-5 VK_norm 6
k = 561 time = 6.29425048828125e-5 VK_n


k = 764 time = 7.104873657226562e-5 VK_norm 6
k = 765 time = 6.699562072753906e-5 VK_norm 6
k = 766 time = 5.698204040527344e-5 VK_norm 6
k = 767 time = 6.580352783203125e-5 VK_norm 8
k = 768 time = 5.698204040527344e-5 VK_norm 8
k = 769 time = 5.698204040527344e-5 VK_norm 8
k = 770 time = 5.698204040527344e-5 VK_norm 8
k = 771 time = 5.602836608886719e-5 VK_norm 8
k = 772 time = 5.5789947509765625e-5 VK_norm 8
k = 773 time = 7.295608520507812e-5 VK_norm 8
k = 774 time = 6.198883056640625e-5 VK_norm 8
k = 775 time = 5.698204040527344e-5 VK_norm 8
k = 776 time = 6.914138793945312e-5 VK_norm 8
k = 777 time = 5.698204040527344e-5 VK_norm 8
k = 778 time = 5.602836608886719e-5 VK_norm 8
k = 779 time = 5.698204040527344e-5 VK_norm 8
k = 780 time = 5.602836608886719e-5 VK_norm 8
k = 781 time = 5.602836608886719e-5 VK_norm 8
k = 782 time = 5.7220458984375e-5 VK_norm 8
k = 783 time = 5.698204040527344e-5 VK_norm 8
k = 784 time = 5.698204040527344e-5 VK_norm 8
k = 785 time = 7.200241088867188e-

...

6.580352783203125e-5 VK_norm 8
k = 10647 time = 6.413459777832031e-5 VK_norm 8
k = 10648 time = 7.081031799316406e-5 VK_norm 8
k = 10649 time = 7.200241088867188e-5 VK_norm 8
k = 10650 time = 7.009506225585938e-5 VK_norm 8
k = 10651 time = 7.104873657226562e-5 VK_norm 8
k = 10652 time = 7.009506225585938e-5 VK_norm 8
k = 10653 time = 6.985664367675781e-5 VK_norm 8
k = 10654 time = 7.104873657226562e-5 VK_norm 8
k = 10655 time = 7.104873657226562e-5 VK_norm 8
k = 10656 time = 7.104873657226562e-5 VK_norm 8
k = 10657 time = 7.104873657226562e-5 VK_norm 8
k = 10658 time = 6.985664367675781e-5 VK_norm 8
k = 10659 time = 7.200241088867188e-5 VK_norm 8
k = 10660 time = 7.200241088867188e-5 VK_norm 8
k = 10661 time = 7.104873657226562e-5 VK_norm 8
k = 10662 time = 6.985664367675781e-5 VK_norm 8
k = 10663 time = 6.794929504394531e-5 VK_norm 8
k = 10664 time = 6.818771362304688e-5 VK_norm 8
k = 10665 time = 6.699562072753906e-5 VK_norm 8
k = 10666 time = 6.699562072753906e-5 VK_norm 8
k = 10667

k = 10725 time = 8.916854858398438e-5 VK_norm 8
k = 10726 time = 9.393692016601562e-5 VK_norm 8
k = 10727 time = 8.296966552734375e-5 VK_norm 8
k = 10728 time = 8.606910705566406e-5 VK_norm 8
k = 10729 time = 6.389617919921875e-5 VK_norm 8
k = 10730 time = 6.29425048828125e-5 VK_norm 8
k = 10731 time = 6.604194641113281e-5 VK_norm 8
k = 10732 time = 7.200241088867188e-5 VK_norm 8
k = 10733 time = 7.295608520507812e-5 VK_norm 8
k = 10734 time = 7.295608520507812e-5 VK_norm 8
k = 10735 time = 7.200241088867188e-5 VK_norm 8
k = 10736 time = 7.319450378417969e-5 VK_norm 8
k = 10737 time = 6.794929504394531e-5 VK_norm 8
k = 10738 time = 7.891654968261719e-5 VK_norm 8
k = 10739 time = 7.486343383789062e-5 VK_norm 8
k = 10740 time = 7.891654968261719e-5 VK_norm 8
k = 10741 time = 7.295608520507812e-5 VK_norm 8
k = 10742 time = 9.298324584960938e-5 VK_norm 8
k = 10743 time = 0.00021314620971679688 VK_norm 8
k = 10744 time = 0.0001659393310546875 VK_norm 8
k = 10745 time = 0.0001499652862548828

Excessive output truncated after 524294 bytes.

10938 time = 6.699562072753906e-5 VK_norm 8
k = 10939 time = 8.082389831542969e-5 VK_norm 8
k = 10940 time = 5.817413330078125e-5 VK_norm 8
k = 10941 time = 5.5789947509765625e-5 VK_norm 8
k = 10942 time = 5.817413330078125e-5 VK_norm 8
k = 10943 time = 5.698204040527344e-5 VK_norm 8
k = 10944 time = 5.793571472167969e-5 VK_norm 8
k = 10945 time = 5.698204040527344e-5 VK_norm 8
k = 10946 time = 5.698204040527344e-5 VK_norm 8
k = 10947 time = 5.698204040527344e-5 VK_norm 8
k = 10948 time = 6.103515625e-5 VK_norm 8
k = 10949 time = 5.698204040527344e-5 VK_norm 8
k = 10950 time = 5.888938903808594e-5 VK_norm 8
k = 10951 time = 5.698204040527344e-5 VK_norm 8
k = 10952 time = 5.7220458984375e-5 VK_norm 8
k = 10953 time = 5.698204040527344e-5 VK_norm 8
k = 10954 time = 5.698204040527344e-5 VK_norm 8
k = 10955 time = 5.698204040527344e-5 VK_norm 8
k = 10956 time = 5.602836608886719e-5 VK_norm 8
k = 10957 time = 5.698204040527344e-5 VK_norm 8
k = 10958 time = 5.698204040527344e-5 VK_norm 8
k =

InterruptException: InterruptException:

In [19]:
gr17_graph, gr17_nodes = graph_from_tsp("../../instances/stsp/gr17.tsp","graphe1")
println("running h_k_algorithm on gr17")
#@time total_distance, one_tree = lkh_subgradient(gr17_graph)

In [20]:
"""Brute force calculates TSP solution"""
function brute_force_tsp(g::Graph, start_node::Any)
    adjacency_list = adjacency_dict(g)
    g_nodes = nodes(g)
    nodes_perm = collect(permutations(collect(1:length(g_nodes))))
    min_dist = Inf
    min_perm = []
    for perm in nodes_perm
        dist = 0
        for i in 1:length(perm)-1
            dist += adjacency_list[perm[i]][perm[i+1]]
        end
        dist += adjacency_list[perm[end]][perm[1]]
        println("permutation", perm)
        println("dist = ", dist)
        if dist < min_dist
            min_dist = dist
            min_perm = perm
        end
        
    end
    return min_dist, min_perm
end
brute_force_tsp(tsp_test, "a")

(24.0, [1, 2, 5, 4, 3])